## 1、agnostic/agnostic_mask

In [14]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import cv2

def get_img_agnostic(img, parse, pose_data):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                    (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                    (parse_array == 12).astype(np.float32) +
                    (parse_array == 16).astype(np.float32) +
                    (parse_array == 17).astype(np.float32) +
                    (parse_array == 18).astype(np.float32) +
                    (parse_array == 19).astype(np.float32))
    
    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)
    agnostic_mask = Image.new('L', (768, 1024), 'white')
    agnostic_mask_draw = ImageDraw.Draw(agnostic_mask)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
    r = int(length_a / 16) + 1
    
    # mask arms
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r*12)
    agnostic_mask_draw.line([tuple(pose_data[i]) for i in [2, 5]], 'gray', width=r * 12)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*6, pointx+r*5, pointy+r*6), 'gray', 'gray')
        agnostic_mask_draw.ellipse((pointx - r * 5, pointy - r * 6, pointx + r * 5, pointy + r * 6), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r*10)
        agnostic_mask_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r * 10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')
        agnostic_mask_draw.ellipse((pointx - r * 5, pointy - r * 5, pointx + r * 5, pointy + r * 5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse((pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'gray', 'gray')
        agnostic_mask_draw.ellipse((pointx - r * 3, pointy - r * 6, pointx + r * 3, pointy + r * 6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')
    agnostic_mask_draw.line([tuple(pose_data[i]) for i in [2, 9]], 'gray', width=r * 6)
    agnostic_mask_draw.line([tuple(pose_data[i]) for i in [5, 12]], 'gray', width=r * 6)
    agnostic_mask_draw.line([tuple(pose_data[i]) for i in [9, 12]], 'gray', width=r * 12)
    agnostic_mask_draw.polygon([tuple(pose_data[i]) for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle((pointx-r*5, pointy-r*9, pointx+r*5, pointy), 'gray', 'gray')
    agnostic_mask_draw.rectangle((pointx - r * 5, pointy - r * 9, pointx + r * 5, pointy), 'gray', 'gray')

    remain_mask = Image.new('L', (768, 1024), 'white')
    for parse_id, pose_ids in [(14, [5, 6, 7]), (15, [2, 3, 4])]:
        mask_arm = Image.new('L', (768, 1024), 'gray')
        mask_arm_draw = ImageDraw.Draw(mask_arm)
        pointx, pointy = pose_data[pose_ids[0]]
        mask_arm_draw.ellipse((pointx - r * 5, pointy - r * 6, pointx + r * 5, pointy + r * 6), 'gray', 'gray')
        for i in pose_ids[1:]:
            if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (
                    pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
                continue
            mask_arm_draw.line([tuple(pose_data[j]) for j in [i - 1, i]], 'gray', width=r * 10)
            pointx, pointy = pose_data[i]
            if i != pose_ids[-1]:
                mask_arm_draw.ellipse((pointx - r * 5, pointy - r * 5, pointx + r * 5, pointy + r * 5), 'gray', 'gray')
        mask_arm_draw.ellipse((pointx - r * 4, pointy - r * 4, pointx + r * 4, pointy + r * 4), 'gray', 'gray')

        parse_arm = (np.array(mask_arm) / 255) * (parse_array == parse_id).astype(np.float32)
        # 如果是gray, 这两句会造成mask叠色
        # agnostic.paste(im, None, Image.fromarray(np.uint8(parse_arm * 255), 'L'))
        # agnostic_mask.paste(remain_mask, None, Image.fromarray(np.uint8(parse_arm * 255), 'L'))

    agnostic.paste(im, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(im, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))
    agnostic_mask.paste(remain_mask, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic_mask.paste(remain_mask, None, Image.fromarray(np.uint8(parse_lower * 255), 'L'))

    return agnostic, agnostic_mask

if __name__ =="__main__":
    data_path = '../VITON-HD/train/'
    output_agnostic_path = '../VITON-HD/train/agnostic'
    output_agnostic_mask_path = '../VITON-HD/train/agnostic_mask'
    
    os.makedirs(output_agnostic_path, exist_ok=True)
    os.makedirs(output_agnostic_mask_path, exist_ok=True)
    
    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):
        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')      
        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print(pose_name)
            continue

        # load parsing image
        im = Image.open(osp.join(data_path, 'image', im_name))
        label_name = im_name.replace('.jpg', '.png')
        im_label = Image.open(osp.join(data_path, 'image-parse-v3', label_name))

        agnostic, agnostic_mask = get_img_agnostic(im, im_label, pose_data)
        
        agnostic.save(osp.join(output_agnostic_path, im_name))
        agnostic_mask.save(osp.join(output_agnostic_mask_path, im_name))

  7%|▋         | 788/11647 [00:44<09:32, 18.97it/s]/tmp/ipykernel_56635/4263028447.py:28: RuntimeWarning: invalid value encountered in true_divide
  pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
/tmp/ipykernel_56635/4263028447.py:29: RuntimeWarning: invalid value encountered in true_divide
  pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
100%|██████████| 11647/11647 [11:11<00:00, 17.33it/s]


## 2、agnostic + warped_cloth merge

融合得到需要修补的边角 inpaint图像

In [2]:
from PIL import Image
import torchvision.transforms as transforms
from os import path as osp
import os
from tqdm import tqdm

def fuse_images(agnostic_path, warped_cloth_path, warped_cloth_mask_path, output_path):
    # 检查输入路径是否存在
    if not osp.exists(agnostic_path) or not osp.exists(warped_cloth_path) or not osp.exists(warped_cloth_mask_path):
        raise ValueError("Input paths do not exist.")

    # 检查输出路径是否存在，如果不存在则创建
    os.makedirs(output_path, exist_ok=True)

    # 获取图像列表
    agnostic_images = [im_name for im_name in os.listdir(agnostic_path) if im_name.endswith(".png") or im_name.endswith(".jpg")]
    warped_cloth_images = [im_name for im_name in os.listdir(warped_cloth_path) if im_name in agnostic_images]
    warped_cloth_mask_images = [im_name for im_name in os.listdir(warped_cloth_mask_path) if im_name in agnostic_images]

    # 确保图像列表一致
    if len(agnostic_images) != len(warped_cloth_images) or len(agnostic_images) != len(warped_cloth_mask_images):
        raise ValueError("Image lists do not match.")

    # 遍历图像列表并进行融合
    for im_name in tqdm(agnostic_images):
        # 加载图像
        agnostic = Image.open(osp.join(agnostic_path, im_name)).convert('RGB')
        warped_cloth = Image.open(osp.join(warped_cloth_path, im_name)).convert('RGB')
        warped_cloth_mask = Image.open(osp.join(warped_cloth_mask_path, im_name)).convert('L')

        # 调整图像大小（如果需要）
        agnostic = agnostic.resize((384, 512))
        warped_cloth = warped_cloth.resize(agnostic.size)
        warped_cloth_mask = warped_cloth_mask.resize(agnostic.size)

        # 转换为 Tensor
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        agnostic = transform(agnostic)
        warped_cloth = transform(warped_cloth)
        warped_cloth_mask = transforms.ToTensor()(warped_cloth_mask)

        warped_cloth = warped_cloth * warped_cloth_mask
        # 融合
        inpaint = agnostic + warped_cloth
        # 反归一化
        inpaint = inpaint * 0.5 + 0.5

        # 将 Tensor 转换为 PIL Image 并保存
        to_pil = transforms.ToPILImage()
        output_filename = osp.join(output_path, im_name)
        to_pil(inpaint).save(output_filename)
        # print(f"Fused image saved to {output_filename}")

if __name__ == "__main__":
    agnostic_path = "../VITON-HD/train/agnostic-v3.2/" 
    warped_cloth_path = "../VITON-HD/train/cloth-warp/" 
    warped_cloth_mask_path = "../VITON-HD/train/cloth-warp-mask/"
    output_path = "./train/inpaint"

    fuse_images(agnostic_path, warped_cloth_path, warped_cloth_mask_path, output_path)

100%|██████████| 11647/11647 [10:57<00:00, 17.71it/s]


## 3、warped_cloth + agnostic_mask

融合得到需要修补的边角 mask

In [3]:
from PIL import Image
import torchvision.transforms as transforms
from os import path as osp
import os
from tqdm import tqdm

def fuse_images(agnostic_mask_path, warped_cloth_mask_path, output_path):
    # 检查输入路径是否存在
    if not osp.exists(agnostic_mask_path) or not osp.exists(warped_cloth_mask_path):
        raise ValueError("Input paths do not exist.")

    # 检查输出路径是否存在，如果不存在则创建
    os.makedirs(output_path, exist_ok=True)

    # 获取图像列表
    agnostic_masks = [im_name for im_name in os.listdir(agnostic_mask_path) if im_name.endswith(".png") or im_name.endswith(".jpg")]
    warped_cloth_masks = [im_name for im_name in os.listdir(warped_cloth_mask_path) if im_name.endswith(".png") or im_name.endswith(".jpg")]

    # 确保图像列表一致
    if len(agnostic_masks) != len(warped_cloth_masks):
        raise ValueError("Image lists do not match.")

    # 遍历图像列表并进行融合
    for im_name in tqdm(agnostic_masks):
        # 加载图像
        agnostic_mask = Image.open(osp.join(agnostic_mask_path, im_name)).convert('L')
        warped_cloth_mask = Image.open(osp.join(warped_cloth_mask_path, im_name.replace('.png', '.jpg'))).convert('L')

        # 调整图像大小（如果需要）
        agnostic_mask = agnostic_mask.resize((384, 512))
        warped_cloth_mask = warped_cloth_mask.resize(agnostic_mask.size)

        agnostic_mask = transforms.ToTensor()(agnostic_mask)
        warped_cloth_mask = 1 - transforms.ToTensor()(warped_cloth_mask)

        inpaint_mask = agnostic_mask * warped_cloth_mask

        # 将 Tensor 转换为 PIL Image 并保存
        to_pil = transforms.ToPILImage()
        output_filename = osp.join(output_path, im_name)
        to_pil(inpaint_mask).save(output_filename)
        # print(f"Fused image saved to {output_filename}")

if __name__ == "__main__":
    agnostic_mask_path = "../VITON-HD/train/inpaint-mask" 
    warped_cloth_mask_path = "../VITON-HD/train/cloth-warp-mask"
    output_path = "./train/inpaint-mask"

    fuse_images(agnostic_mask_path, warped_cloth_mask_path, output_path)

100%|██████████| 11647/11647 [03:27<00:00, 56.19it/s]
